In [34]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
#from time import time
#from torch import optim

In [35]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, 1)))
    y_ = torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    conditionnumber=np.linalg.cond(X2_.numpy())
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg,conditionnumber

In [36]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [37]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [38]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [56]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(20,21,100): 
        for num_X2feature in [2]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1,conditionnumber)=generatedata(noise=0.1)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.0015
                    results_S=[]
                    t=0
                    before1=0
                    while True:                     
                        Y1=y
                        Y2=torch.exp(w2[0]*X2)*torch.sin(w2[1]*X2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'={:.4f}'.format(Loss.data))
                        #print('Loss',t,'=',Loss)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-5:
                            break
                        before1=Loss
                        if t>=260:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    #results_error.append(error_each)
                    error_each2=(torch.norm(w2-true_w2))/torch.norm(true_w2)
                    
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                #print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')
                print(num_X2feature,num_example,'绝对误差',error_each,end='   ')
                print('相对误差',error_each2,end='   ')
                #print('真实置换矩阵为：',true_P)    
                
                error_reg2=(torch.norm(y_-torch.exp(w2[0]*torch.mm(S,X2_))*torch.sin(w2[1]*torch.mm(S,X2_)))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2,end='   ')
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print('置换矩阵误差',error_P,end='   ')
                matrix_count=0
                for i_2 in range(num_example):
                    for j_2 in range(num_example):
                        if (abs(S[i_2][j_2]-1)<0.02):
                            matrix_count+=1
                print('离置换矩阵距离',matrix_count,end='   ')
                print('条件数',conditionnumber)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 =58.4352
#Loss 1 =57.6237
#Loss 2 =52.1555
#Loss 3 =47.3666
#Loss 4 =42.9621
#Loss 5 =38.6990
#Loss 6 =34.3975
#Loss 7 =29.9416
#Loss 8 =25.2811
#Loss 9 =20.4406
#Loss 10 =15.5463
#Loss 11 =10.8593
#Loss 12 =6.7887
#Loss 13 =3.7723
#Loss 14 =1.9951
#Loss 15 =1.1944
#Loss 16 =0.8908
#Loss 17 =0.7547
#Loss 18 =0.6633
#Loss 19 =0.5867
#Loss 20 =0.5189
#Loss 21 =0.4585
#Loss 22 =0.4050
#Loss 23 =0.3578
#Loss 24 =0.3162
#Loss 25 =0.2797
#Loss 26 =0.2479
#Loss 27 =0.2202
#Loss 28 =0.1963
#Loss 29 =0.1756
#Loss 30 =0.1577
#Loss 31 =0.1424
#Loss 32 =0.1292
#Loss 33 =0.1180
#Loss 34 =0.1085
#Loss 35 =0.1004
#Loss 36 =0.0935
#Loss 37 =0.0878
#Loss 38 =0.0829
#Loss 39 =0.0788
#Loss 40 =0.0754
#Loss 41 =0.0725
#Loss 42 =0.0701
#Loss 43 =0.0681
#Loss 44 =0.0664
#Loss 45 =0.0650
#Loss 46 =0.0639
#Loss 47 =0.0629
#Loss 48 =0.0621
#Loss 49 =0.0614
#Loss 50 =0.0609
#Loss 51 =0.0604
#Loss 52 =0.0600
#Loss 53 =0.0597
#Loss 54 =0.0595
#Loss 55 =0.0593
#Loss 56 =0.0591
#Loss 57 =0.0589
#Loss 58 =0.0

#Loss 56 =2.9306
#Loss 57 =2.9269
#Loss 58 =2.9234
#Loss 59 =2.9200
#Loss 60 =2.9167
#Loss 61 =2.9136
#Loss 62 =2.9106
#Loss 63 =2.9078
#Loss 64 =2.9050
#Loss 65 =2.9024
#Loss 66 =2.8999
#Loss 67 =2.8975
#Loss 68 =2.8952
#Loss 69 =2.8930
#Loss 70 =2.8909
#Loss 71 =2.8889
#Loss 72 =2.8870
#Loss 73 =2.8851
#Loss 74 =2.8834
#Loss 75 =2.8817
#Loss 76 =2.8801
#Loss 77 =2.8786
#Loss 78 =2.8771
#Loss 79 =2.8758
#Loss 80 =2.8744
#Loss 81 =2.8732
#Loss 82 =2.8720
#Loss 83 =2.8708
#Loss 84 =2.8697
#Loss 85 =2.8687
#Loss 86 =2.8677
#Loss 87 =2.8667
#Loss 88 =2.8658
#Loss 89 =2.8649
#Loss 90 =2.8641
#Loss 91 =2.8632
#Loss 92 =2.8624
#Loss 93 =2.8617
#Loss 94 =2.8610
#Loss 95 =2.8603
#Loss 96 =2.8596
#Loss 97 =2.8590
#Loss 98 =2.8584
#Loss 99 =2.8579
#Loss 100 =2.8574
#Loss 101 =2.8568
#Loss 102 =2.8564
#Loss 103 =2.8559
#Loss 104 =2.8555
#Loss 105 =2.8551
#Loss 106 =2.8547
#Loss 107 =2.8543
#Loss 108 =2.8539
#Loss 109 =2.8536
#Loss 110 =2.8533
#Loss 111 =2.8530
#Loss 112 =2.8527
#Loss 113 =2.8524


#Loss 60 =199.2412
#Loss 61 =199.1948
#Loss 62 =199.1531
#Loss 63 =199.1160
#Loss 64 =199.0832
#Loss 65 =199.0541
#Loss 66 =199.0285
#Loss 67 =199.0059
#Loss 68 =198.9860
#Loss 69 =198.9684
#Loss 70 =198.9527
#Loss 71 =198.9389
#Loss 72 =198.9268
#Loss 73 =198.9162
#Loss 74 =198.9070
#Loss 75 =198.8989
#Loss 76 =198.8919
#Loss 77 =198.8858
#Loss 78 =198.8804
#Loss 79 =198.8756
#Loss 80 =198.8709
#Loss 81 =198.8668
#Loss 82 =198.8633
#Loss 83 =198.8602
#Loss 84 =198.8575
#Loss 85 =198.8552
#Loss 86 =198.8531
#Loss 87 =198.8514
 2 20 绝对误差 tensor(3.6094, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.7278, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0241, dtype=torch.float64)   实验回归误差 tensor(0.8890, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 20   条件数 1.0
Loss 0 =37.8434
#Loss 1 =37.6363
#Loss 2 =35.1973
#Loss 3 =33.0268
#Loss 4 =31.0777
#Loss 5 =29.2983
#Loss 6 =27.6404
#Loss

#Loss 168 =0.1059
#Loss 169 =0.1059
#Loss 170 =0.1059
#Loss 171 =0.1059
#Loss 172 =0.1059
#Loss 173 =0.1059
#Loss 174 =0.1059
#Loss 175 =0.1059
#Loss 176 =0.1059
#Loss 177 =0.1059
#Loss 178 =0.1059
#Loss 179 =0.1059
#Loss 180 =0.1059
#Loss 181 =0.1059
#Loss 182 =0.1059
#Loss 183 =0.1059
#Loss 184 =0.1059
#Loss 185 =0.1059
#Loss 186 =0.1059
#Loss 187 =0.1058
#Loss 188 =0.1058
#Loss 189 =0.1058
#Loss 190 =0.1058
#Loss 191 =0.1058
#Loss 192 =0.1058
#Loss 193 =0.1058
#Loss 194 =0.1058
#Loss 195 =0.1058
#Loss 196 =0.1058
#Loss 197 =0.1058
#Loss 198 =0.1058
#Loss 199 =0.1058
#Loss 200 =0.1058
#Loss 201 =0.1058
#Loss 202 =0.1058
#Loss 203 =0.1058
#Loss 204 =0.1058
#Loss 205 =0.1058
#Loss 206 =0.1058
#Loss 207 =0.1058
#Loss 208 =0.1058
 2 20 绝对误差 tensor(0.0174, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(0.0168, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0849, dtype=torch.float64)   实验回归误差 tensor(0.0652, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 

#Loss 19 =0.9177
#Loss 20 =0.8412
#Loss 21 =0.7713
#Loss 22 =0.7076
#Loss 23 =0.6497
#Loss 24 =0.5971
#Loss 25 =0.5495
#Loss 26 =0.5065
#Loss 27 =0.4676
#Loss 28 =0.4328
#Loss 29 =0.4015
#Loss 30 =0.3735
#Loss 31 =0.3486
#Loss 32 =0.3264
#Loss 33 =0.3067
#Loss 34 =0.2892
#Loss 35 =0.2738
#Loss 36 =0.2601
#Loss 37 =0.2481
#Loss 38 =0.2375
#Loss 39 =0.2282
#Loss 40 =0.2199
#Loss 41 =0.2127
#Loss 42 =0.2062
#Loss 43 =0.2005
#Loss 44 =0.1954
#Loss 45 =0.1909
#Loss 46 =0.1869
#Loss 47 =0.1832
#Loss 48 =0.1799
#Loss 49 =0.1769
#Loss 50 =0.1742
#Loss 51 =0.1717
#Loss 52 =0.1693
#Loss 53 =0.1672
#Loss 54 =0.1652
#Loss 55 =0.1633
#Loss 56 =0.1615
#Loss 57 =0.1598
#Loss 58 =0.1582
#Loss 59 =0.1567
#Loss 60 =0.1553
#Loss 61 =0.1539
#Loss 62 =0.1525
#Loss 63 =0.1513
#Loss 64 =0.1500
#Loss 65 =0.1488
#Loss 66 =0.1477
#Loss 67 =0.1465
#Loss 68 =0.1455
#Loss 69 =0.1444
#Loss 70 =0.1434
#Loss 71 =0.1424
#Loss 72 =0.1414
#Loss 73 =0.1405
#Loss 74 =0.1396
#Loss 75 =0.1387
#Loss 76 =0.1378
#Loss 77 =0.13

...

#Loss 67 =0.3325
#Loss 68 =0.3286
#Loss 69 =0.3247
#Loss 70 =0.3210
#Loss 71 =0.3174
#Loss 72 =0.3139
#Loss 73 =0.3106
#Loss 74 =0.3073
#Loss 75 =0.3041
#Loss 76 =0.3010
#Loss 77 =0.2981
#Loss 78 =0.2952
#Loss 79 =0.2923
#Loss 80 =0.2896
#Loss 81 =0.2870
#Loss 82 =0.2844
#Loss 83 =0.2819
#Loss 84 =0.2795
#Loss 85 =0.2772
#Loss 86 =0.2749
#Loss 87 =0.2727
#Loss 88 =0.2705
#Loss 89 =0.2685
#Loss 90 =0.2664
#Loss 91 =0.2645
#Loss 92 =0.2626
#Loss 93 =0.2607
#Loss 94 =0.2589
#Loss 95 =0.2572
#Loss 96 =0.2555
#Loss 97 =0.2539
#Loss 98 =0.2523
#Loss 99 =0.2507
#Loss 100 =0.2492
#Loss 101 =0.2478
#Loss 102 =0.2464
#Loss 103 =0.2450
#Loss 104 =0.2437
#Loss 105 =0.2424
#Loss 106 =0.2411
#Loss 107 =0.2399
#Loss 108 =0.2387
#Loss 109 =0.2375
#Loss 110 =0.2364
#Loss 111 =0.2353
#Loss 112 =0.2343
#Loss 113 =0.2333
#Loss 114 =0.2323
#Loss 115 =0.2313
#Loss 116 =0.2304
#Loss 117 =0.2294
#Loss 118 =0.2286
#Loss 119 =0.2277
#Loss 120 =0.2269
#Loss 121 =0.2260
#Loss 122 =0.2253
#Loss 123 =0.2245
#Loss 1

#Loss 49 =1.1484
#Loss 50 =1.1460
#Loss 51 =1.1437
#Loss 52 =1.1416
#Loss 53 =1.1396
#Loss 54 =1.1376
#Loss 55 =1.1358
#Loss 56 =1.1341
#Loss 57 =1.1325
#Loss 58 =1.1310
#Loss 59 =1.1295
#Loss 60 =1.1281
#Loss 61 =1.1268
#Loss 62 =1.1255
#Loss 63 =1.1243
#Loss 64 =1.1232
#Loss 65 =1.1221
#Loss 66 =1.1211
#Loss 67 =1.1202
#Loss 68 =1.1192
#Loss 69 =1.1184
#Loss 70 =1.1175
#Loss 71 =1.1168
#Loss 72 =1.1160
#Loss 73 =1.1153
#Loss 74 =1.1146
#Loss 75 =1.1140
#Loss 76 =1.1134
#Loss 77 =1.1128
#Loss 78 =1.1122
#Loss 79 =1.1117
#Loss 80 =1.1112
#Loss 81 =1.1108
#Loss 82 =1.1103
#Loss 83 =1.1099
#Loss 84 =1.1095
#Loss 85 =1.1091
#Loss 86 =1.1088
#Loss 87 =1.1084
#Loss 88 =1.1081
#Loss 89 =1.1078
#Loss 90 =1.1075
#Loss 91 =1.1072
#Loss 92 =1.1069
#Loss 93 =1.1067
#Loss 94 =1.1064
#Loss 95 =1.1062
#Loss 96 =1.1060
#Loss 97 =1.1058
#Loss 98 =1.1056
#Loss 99 =1.1054
#Loss 100 =1.1052
#Loss 101 =1.1051
#Loss 102 =1.1049
#Loss 103 =1.1048
#Loss 104 =1.1046
#Loss 105 =1.1045
#Loss 106 =1.1044
#Loss 1

#Loss 64 =1.3842
#Loss 65 =1.3761
#Loss 66 =1.3680
#Loss 67 =1.3601
#Loss 68 =1.3523
#Loss 69 =1.3446
#Loss 70 =1.3371
#Loss 71 =1.3296
#Loss 72 =1.3222
#Loss 73 =1.3149
#Loss 74 =1.3078
#Loss 75 =1.3007
#Loss 76 =1.2937
#Loss 77 =1.2868
#Loss 78 =1.2800
#Loss 79 =1.2733
#Loss 80 =1.2666
#Loss 81 =1.2601
#Loss 82 =1.2536
#Loss 83 =1.2472
#Loss 84 =1.2409
#Loss 85 =1.2347
#Loss 86 =1.2285
#Loss 87 =1.2224
#Loss 88 =1.2164
#Loss 89 =1.2104
#Loss 90 =1.2046
#Loss 91 =1.1987
#Loss 92 =1.1930
#Loss 93 =1.1873
#Loss 94 =1.1817
#Loss 95 =1.1762
#Loss 96 =1.1707
#Loss 97 =1.1652
#Loss 98 =1.1599
#Loss 99 =1.1545
#Loss 100 =1.1493
#Loss 101 =1.1441
#Loss 102 =1.1389
#Loss 103 =1.1339
#Loss 104 =1.1288
#Loss 105 =1.1238
#Loss 106 =1.1189
#Loss 107 =1.1140
#Loss 108 =1.1092
#Loss 109 =1.1044
#Loss 110 =1.0996
#Loss 111 =1.0949
#Loss 112 =1.0903
#Loss 113 =1.0857
#Loss 114 =1.0811
#Loss 115 =1.0766
#Loss 116 =1.0722
#Loss 117 =1.0677
#Loss 118 =1.0633
#Loss 119 =1.0590
#Loss 120 =1.0547
#Loss 121 

#Loss 220 =0.6749
#Loss 221 =0.6745
#Loss 222 =0.6740
#Loss 223 =0.6736
#Loss 224 =0.6731
#Loss 225 =0.6727
#Loss 226 =0.6723
#Loss 227 =0.6718
#Loss 228 =0.6714
#Loss 229 =0.6710
#Loss 230 =0.6705
#Loss 231 =0.6701
#Loss 232 =0.6697
#Loss 233 =0.6692
#Loss 234 =0.6688
#Loss 235 =0.6684
#Loss 236 =0.6680
#Loss 237 =0.6676
#Loss 238 =0.6671
#Loss 239 =0.6667
#Loss 240 =0.6663
#Loss 241 =0.6659
#Loss 242 =0.6655
#Loss 243 =0.6650
#Loss 244 =0.6646
#Loss 245 =0.6642
#Loss 246 =0.6638
#Loss 247 =0.6634
#Loss 248 =0.6630
#Loss 249 =0.6626
#Loss 250 =0.6622
#Loss 251 =0.6618
#Loss 252 =0.6614
#Loss 253 =0.6610
#Loss 254 =0.6606
#Loss 255 =0.6602
#Loss 256 =0.6598
#Loss 257 =0.6594
#Loss 258 =0.6590
#Loss 259 =0.6586
#Loss 260 =0.6582
超过迭代上限
 2 20 绝对误差 tensor(1.8849, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.3015, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.2402, dtype=torch.float64)   实验回归误差 tensor(0.4635, dtype=torch.float64, grad_fn=<DivBackward0>)   

#Loss 126 =4.2924
#Loss 127 =7.0198
#Loss 128 =4.2923
#Loss 129 =7.0197
#Loss 130 =4.2923
#Loss 131 =7.0196
#Loss 132 =4.2923
#Loss 133 =7.0196
#Loss 134 =4.2923
#Loss 135 =7.0197
#Loss 136 =4.2923
#Loss 137 =7.0197
#Loss 138 =4.2923
#Loss 139 =7.0197
#Loss 140 =4.2923
#Loss 141 =7.0196
#Loss 142 =4.2923
#Loss 143 =7.0197
#Loss 144 =4.2923
#Loss 145 =7.0197
#Loss 146 =4.2923
#Loss 147 =7.0197
#Loss 148 =4.2923
#Loss 149 =7.0197
#Loss 150 =4.2923
#Loss 151 =7.0197
#Loss 152 =4.2923
#Loss 153 =7.0197
#Loss 154 =4.2923
#Loss 155 =7.0197
#Loss 156 =4.2923
#Loss 157 =7.0197
#Loss 158 =4.2923
#Loss 159 =7.0197
#Loss 160 =4.2923
#Loss 161 =7.0197
#Loss 162 =4.2923
#Loss 163 =7.0197
#Loss 164 =4.2923
#Loss 165 =7.0197
#Loss 166 =4.2923
#Loss 167 =7.0197
#Loss 168 =4.2923
#Loss 169 =7.0197
#Loss 170 =4.2923
#Loss 171 =7.0197
#Loss 172 =4.2923
#Loss 173 =7.0197
#Loss 174 =4.2923
#Loss 175 =7.0197
#Loss 176 =4.2923
#Loss 177 =7.0197
#Loss 178 =4.2923
#Loss 179 =7.0197
#Loss 180 =4.2923
#Loss 181 